From the example at https://huggingface.co/facebook/mgenre-wiki

paper https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00460/110051/Multilingual-Autoregressive-Entity-Linking


## install

get the genre code from https://github.com/facebookresearch/GENRE 
then do pip3 install . 

and do pip3 install marisa-trie


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()

# from genre.trie import Trie, MarisaTrie
# import pickle 

#with open("mGENRE/titles_lang_all105_marisa_trie_with_redirect.pkl", "rb") as f:
#    trie = pickle.load(f)

/home/p107437/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
#sentences = ["[START] Einstein [END] era un fisico tedesco."]
# sentences = ["Ze was getrouwd met [BEGIN] Ron Graham [END] ."]
sentences = ["De wedstrijd kende een finale rijk aan schermutselingen, tot uiteindelijk [START} Stijn Devolder [END], vorige week reeds 3e in de Ronde van Zwitserland en dus in bloedvorm, er op 9 km van de streep vandoor ging."]
sentences = tokenizer(sentences, return_tensors="pt")

constrained beam search still does not work, trie gives an error...

In [12]:
outputs = model.generate(
    **sentences, 
    num_beams=5,
    num_return_sequences=5,
    # OPTIONAL: use constrained beam search
    # prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
)

In [17]:
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(results)

['Stijn Devolder >> nl', 'Stijn Devolder (wielrenner) >> nl', 'Stijn De Volder >> nl', 'Stijn Devolder (wielrenner, 1991) >> nl', 'Stijn Devolder (wielrenner, 1970) >> nl']


In [28]:
import requests

S = requests.Session()

URL = "https://nl.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "prop": "pageprops",
    "format": "json"
}

def find_qid(results) :
    qid = 0
    for result in results:
        if result.endswith('nl') and not(qid) : 
            label = result.split(' >> ')[0]
            PARAMS['titles'] = label
            Wikipedia_data = S.get(url=URL, params=PARAMS).json()
            Pages = Wikipedia_data['query']['pages']
            for (key,val) in Pages.items() : 
                try : 
                    qid = val['pageprops']['wikibase_item']
                    break
                except : 
                    True
    return(qid)

In [29]:
find_qid(results)

'Q382383'